In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import nept

from matplotlib import animation
from IPython.display import HTML

from startup import extract_xy, sort_led_locations, correct_targets, median_filter, remove_jumps_to_feeder
from loading_data import get_data, unzip_nvt_file, zip_nvt_file
from analyze_decode_bytrial import get_trials

import warnings
warnings.filterwarnings("ignore")

In [ ]:
thisdir = os.getcwd()
dataloc = os.path.join(thisdir, 'cache', 'data')
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots", "correcting_position")

In [ ]:
# import info.r066d2 as info
import info.r063d8 as info

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)

In [ ]:
filename = os.path.join(dataloc, info.position_filename)
variance_thresh = 5.
epsilon = 0.01

In [ ]:
# Load raw position from file
nvt_data = nept.load_nvt(filename)
targets = nvt_data['targets']
times = nvt_data['time']

In [ ]:
# Initialize x, y arrays
x = np.zeros(targets.shape)
y = np.zeros(targets.shape)

# X and Y are stored in a custom bitfield. See Neuralynx data file format documentation for details.
# Briefly, each record contains up to 50 targets, each stored in 32bit field.
# X field at [20:31] and Y at [4:15].
# TODO: make into a separate function in nept
for target in range(targets.shape[1]):
    this_sample = targets[:, target]
    for sample in range(targets.shape[0]):
        # When the bitfield is equal to zero there is no valid data for that field
        # and remains zero for the rest of the bitfields in the record.
        if this_sample[target] == 0:
            break
        x[sample, target], y[sample, target] = extract_xy(int(this_sample[sample]))

# Replacing targets with no samples with nan instead of 0
x[x == 0] = np.nan
y[y == 0] = np.nan

# Scale the positions
x = x / info.scale_targets[0]
y = y / info.scale_targets[1]

In [ ]:
plt.plot(times, y, "k.", ms=4)
plt.show()

In [ ]:
# Finding which feeder led is on over time
leds = []
leds.extend([(event, 'led1') for event in events['led1']])
leds.extend([(event, 'led2') for event in events['led2']])
sorted_leds = sorted(leds)

# Get an array of feeder locations when that feeder is actively flashing
feeder_x_location = np.empty(times.shape[0]) * np.nan
feeder_y_location = np.empty(times.shape[0]) * np.nan

n_ledon = 6
for time, label in sorted_leds:
    idx = nept.find_nearest_idx(times, time)
    x_location = info.path_pts['feeder1'][0] if label == 'led1' else info.path_pts['feeder2'][0]
    y_location = info.path_pts['feeder1'][1] if label == 'led1' else info.path_pts['feeder2'][1]

    feeder_x_location[idx:idx+n_ledon] = x_location
    feeder_y_location[idx:idx+n_ledon] = y_location

In [ ]:
# Removing the contaminated samples that are closest to the feeder location
def remove_feeder_contamination(original_targets, current_feeder, dist_to_feeder=5):
    targets = np.array(original_targets)
    for i, (target, feeder) in enumerate(zip(targets, current_feeder)):
        if not np.isnan(feeder):
            dist = np.abs(target - feeder) < dist_to_feeder
            target[dist] = np.nan
        targets[i] = target
    return targets

x = remove_feeder_contamination(x, feeder_x_location)
y = remove_feeder_contamination(y, feeder_y_location)

In [ ]:
plt.plot(times, y, "k.", ms=4)
plt.show()

In [ ]:
# Removing the problem samples that are furthest from the previous location
def remove_based_on_std(original_targets, std_thresh=10):
    targets = np.array(original_targets)
    stds = np.nanstd(targets, axis=1)[:, np.newaxis]
    
    problem_samples = np.where(stds > std_thresh)[0]
    
    for i in problem_samples:
        idx = np.nanargmax(np.abs(targets[i] - np.nanmean(targets[i-1])))
        targets[i][idx] = np.nan
    
    return targets

x = remove_based_on_std(x)
y = remove_based_on_std(y)

In [ ]:
# Calculating the mean of the remaining targets
xx = np.nanmean(x, axis=1)
yy = np.nanmean(y, axis=1)
ttimes = times

In [ ]:
plt.plot(ttimes, yy, "k.", ms=4)
plt.show()

In [ ]:
len(yy)

In [ ]:
# Apply a median filter
xx, yy = median_filter(xx, yy)

In [ ]:
# Construct a nept.Position object
position = nept.Position(np.hstack(np.array([xx, yy])[..., np.newaxis]), ttimes)

In [ ]:
plt.plot(position.x, position.y, "k.", ms=1)
plt.show()

In [ ]:
plt.plot(position.time, position.y, "k.", ms=4)
plt.show()

In [ ]:
check_start = 60000
check_end = 70000

plt.plot(position.time[check_start:check_end], position.y[check_start:check_end], "k.", ms=5)
plt.ylim(20,70)
plt.show()

In [ ]:
trial_epochs = get_trials(events, info.task_times["phase3"])
trial_idx = 2
start = trial_epochs[trial_idx].start
stop = trial_epochs[trial_idx].stop

In [ ]:
trial = position.time_slice(start, stop)
plt.plot(trial.x, trial.y, "k.")
plt.show()

In [ ]:
# Determining the percent of samples used
position.n_samples / len(targets) * 100